In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB


def solve_relaxation(X, u_lb, u_ub, v_lb, v_ub):
    m, n = X.shape
    model = gp.Model()
    

    # Variables
    u = model.addVars(m, lb=u_lb, ub=u_ub)
    v = model.addVars(n, lb=v_lb, ub=v_ub)
    z = model.addVars(m, n)
    t = model.addVars(m, n, lb=0)

    # Linearization constraints
    for i in range(m):
        for j in range(n):
            model.addConstr(u_ub[i] * v_ub[j] - v_ub[j] * u[i] - u_ub[i] * v[j] + z[i, j] >= 0)
            model.addConstr(-u_ub[i] * v_lb[j] + v_lb[j] * u[i] + u_ub[i] * v[j] - z[i, j] >= 0)
            model.addConstr(-u_lb[i] * v_ub[j] + v_ub[j] * u[i] + u_lb[i] * v[j] - z[i, j] >= 0)
            model.addConstr(u_lb[i] * v_lb[j] - v_lb[j] * u[i] - u_lb[i] * v[j] + z[i, j] >= 0)

    # Absolute error
    for i in range(m):
        for j in range(n):
            model.addConstr(t[i, j] >= X[i, j] - z[i, j])
            model.addConstr(t[i, j] >= z[i, j] - X[i, j])

    model.setObjective(gp.quicksum(t[i, j] for i in range(m) for j in range(n)), GRB.MINIMIZE)
    model.optimize()

    if model.status == GRB.OPTIMAL:
        u_val = np.array([u[i].X for i in range(m)])
        v_val = np.array([v[j].X for j in range(n)])
        return model.ObjVal, u_val, v_val
    else:
        return np.inf, None, None




In [2]:
def rank1l1NMF(X, delta):
    m, n = X.shape
    X_nnz = np.sum(X)

    u_lb, u_ub = np.zeros(m), np.ones(m)
    v_lb, v_ub = np.zeros(n), np.full(n, X_nnz + 1)

    best_u, best_v = None, None
    best_obj = np.inf
    best_bounds = None

    stack = [(u_lb.copy(), u_ub.copy(), v_lb.copy(), v_ub.copy())]

    while stack:
        ulb, uub, vlb, vub = stack.pop()

        # Bounding box too small?
        if max(uub - ulb) <= delta and max(vub - vlb) <= delta:
            obj, u_val, v_val = solve_relaxation(X, ulb, uub, vlb, vub)
            if obj < best_obj:
                best_obj = obj
                best_u, best_v = u_val, v_val
                best_bounds = (ulb.copy(), uub.copy(), vlb.copy(), vub.copy())
            continue

        # Solve LP relaxation
        obj, u_val, v_val = solve_relaxation(X, ulb, uub, vlb, vub)
        if obj >= best_obj:
            continue

        # Branching
        if max(uub - ulb) >= max(vub - vlb):
            i = np.argmax(uub - ulb)
            mid = (ulb[i] + uub[i]) / 2
            ulb1, uub1 = ulb.copy(), uub.copy()
            ulb2, uub2 = ulb.copy(), uub.copy()
            uub1[i], ulb2[i] = mid, mid
            stack.append((ulb1, uub1, vlb.copy(), vub.copy()))
            stack.append((ulb2, uub2, vlb.copy(), vub.copy()))
        else:
            j = np.argmax(vub - vlb)
            mid = (vlb[j] + vub[j]) / 2
            vlb1, vub1 = vlb.copy(), vub.copy()
            vlb2, vub2 = vlb.copy(), vub.copy()
            vub1[j], vlb2[j] = mid, mid
            stack.append((ulb.copy(), uub.copy(), vlb1, vub1))
            stack.append((ulb.copy(), uub.copy(), vlb2, vub2))

    if best_u is not None and best_bounds is not None:
        ulb_best, uub_best, vlb_best, vub_best = best_bounds
        return best_u, best_v, ulb_best, uub_best, vlb_best, vub_best
    else:
        return None, None, None, None, None, None


In [ ]:
import numpy as np

X = np.array([
    [1, 1, 0],
    [1, 0, 0],
    [0, 1, 1]
])

delta = 0.1  # précision du branch & bound

u, v, u_min, u_max, v_min, v_max = rank1l1NMF(X, delta)

print("u =", np.round(u, 3))
print("v =", np.round(v, 3))
print("u @ v.T =\n", np.round(np.outer(u, v), 2))
print("||X - uvᵗ||₁ =", np.sum(np.abs(X - np.outer(u, v))))


Set parameter Username
Set parameter LicenseID to value 2661355


Academic license - for non-commercial use only - expires 2026-05-06
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-1035G4 CPU @ 1.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 54 rows, 24 columns and 108 nonzeros
Model fingerprint: 0xec58846e
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 17 rows and 4 columns
Presolve time: 0.03s
Presolved: 37 rows, 20 columns, 83 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.500000e+00   0.000000e+00      0s
      11    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.05 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Gurobi Optimizer version 12.0.1 build 